# Classification of text review as positive or negative of imdb 

In [2]:
# all libraries
import numpy as np

import tensorflow as tf

!pip install tensorflow-hub
!pip install tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.7 MB 3.2 MB/s eta 0:00:01
   - -------------------------------------- 0.1/1.7 MB 787.7 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.7 MB 944.1 kB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.7 MB 1.2 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/1.7 MB 1.0 MB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.7 MB 1.1 MB/s eta 0:00:02
   -------- ------------------------------- 0.3/1.7 MB 1.1 MB/s eta 0:00:02
   --------- ------------------------------ 0.4/1.7 MB 1.1 MB/s eta 0:00:02
   ---------- ----------------------------- 0.5/1.7 MB 1.2 MB/s eta 0:00:02
   ------------ --------------------------- 0.5/1.7 MB 1.1 MB/s eta 0:00:02
   ------------ ------


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/5.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.4 MB 1.9 MB/s eta 0:00:03
   ---------------------------------------- 0.0/5.4 MB 1.9 MB/s eta 0:00:03
    --------------------------------------- 0.1/5.4 MB 558.5 kB/s eta 0:00:10
    --------------------------------------- 0.1/5.4 MB 653.6 kB/s eta 0:00:09
    --------------------------------------- 0.1/5.4 MB 653.6 kB/s eta 0:00:09
   - -------------------------------------- 0.2/5.4 MB 653.6 kB/s eta 0:00:08
   - -------------------------------------- 0.2/5.4 MB 692.4 kB/s eta 0:00:08
   - -------------------------------------- 0.2/5.4 MB 686.8 kB/s eta 0:00:08
   -- ------------------------------------- 0.3/5.4 MB 681.0 kB/s eta 0:00:08
   -- ------------------------------------- 0.3/5.4 MB 654.6 kB/s eta 0:00:08
   -- ------------------------------------- 0.3/5.4 M


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Version:  2.12.0
Eager mode:  True
Hub version:  0.16.1
GPU is NOT AVAILABLE


In [3]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\HARSHIT GOYAL\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteBOYDUF\imdb_revie…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\HARSHIT GOYAL\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteBOYDUF\imdb_revie…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\HARSHIT GOYAL\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteBOYDUF\imdb_revie…

Dataset imdb_reviews downloaded and prepared to C:\Users\HARSHIT GOYAL\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [4]:
# lets see what is inside in the data
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [5]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [6]:
# Buliding the model
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [8]:
# compiling the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [9]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 57ms/step - loss: 0.9810 - accuracy: 0.5073 - val_loss: 0.8225 - val_accuracy: 0.5085
Epoch 2/20
30/30 [==============================] - 1s 49ms/step - loss: 0.7520 - accuracy: 0.5376 - val_loss: 0.7083 - val_accuracy: 0.5666
Epoch 3/20
30/30 [==============================] - 1s 49ms/step - loss: 0.6720 - accuracy: 0.5865 - val_loss: 0.6457 - val_accuracy: 0.5970
Epoch 4/20
30/30 [==============================] - 1s 49ms/step - loss: 0.6143 - accuracy: 0.6309 - val_loss: 0.5977 - val_accuracy: 0.6422
Epoch 5/20
30/30 [==============================] - 1s 48ms/step - loss: 0.5660 - accuracy: 0.6771 - val_loss: 0.5573 - val_accuracy: 0.6879
Epoch 6/20
30/30 [==============================] - 1s 48ms/step - loss: 0.5227 - accuracy: 0.7179 - val_loss: 0.5219 - val_accuracy: 0.7170
Epoch 7/20
30/30 [==============================] - 1s 48ms/step - loss: 0.4826 - accuracy: 0.7540 - val_loss: 0.4893 - val_accuracy: 0.7501
Epoch 8/20
30

In [10]:
# evaluate
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 1s - loss: 0.3248 - accuracy: 0.8567 - 1s/epoch - 24ms/step
loss: 0.325
accuracy: 0.857
